In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/openai/CLIP.git transformers bitsandbytes

     |████████████████████████████████| 5.8 MB 15.0 MB/s 
     |████████████████████████████████| 62.5 MB 1.1 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 182 kB 74.3 MB/s 
     |████████████████████████████████| 7.6 MB 61.5 MB/s 


In [ ]:
!cp -r /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/data /content/
!unzip -q /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/videos_train.zip 
!mv videos_train /content/data

!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/config.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/utils.py /content/
!cp -i /content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/model.py /content/

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm, trange
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
#from torch.cuda.amp import autocast
import io
import os
import PIL
import random
import numpy as np
import torch
import torchvision
import transformers
import more_itertools
import numpy as np
import matplotlib.pyplot as plt
#from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
#from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import clip

import transformers

from utils import *
from model import *
import re

import warnings
warnings.simplefilter('ignore')

import time

In [ ]:
def get_caption(prefix, prompt=''):
        prefix = prefix.to(device)
        display(prefix.shape)
        with torch.no_grad():

            prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

            start = time.time()

            if prompt:
                generated_text_prefix = generate2(model, tokenizer, prompt=prompt, embed=prefix_embed)
            else:
                generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)

            print(time.time() - start)

        return generated_text_prefix.replace('\n',' ').replace('<|endoftext|','')

def get_ans(clip_emb, prompt):
        output = get_caption(clip_emb, prompt=prompt)
        ans = output[len(prompt):].strip()
        return {'answer': ans}


# parser = argparse.ArgumentParser()
# parser.add_argument('--input_path', default='./input_test.csv', type=str, help='input path')
# parser.add_argument('--video_path', default='./videos_val/', type=str, help='input path')
# parser.add_argument('--output_path', default='./output/', type=str, help='config path')
# args = parser.parse_args()

config = dict(
        model_path = '/content/drive/MyDrive/Olimpiads/nto_hack_2022/V2/v2_1.pt',
        video_path = '/content/data/videos_train/',
        val_path = '/content/data/new_valid.csv',
        gpt = 'gpt2',
        prefix_len = 40
    )


prefix_length = config['prefix_len']#40

device = 'cuda'
clip_model, preprocess = clip.load("ViT-L/14@336px", device=device, jit=False)
clip_model.to(device)

tokenizer = GPT2Tokenizer.from_pretrained(config['gpt'])


model_path = config['gpt']
model = ClipCaptionModel(prefix_length = prefix_length, backbone = config['gpt'])

model.load_state_dict(torch.load(config['model_path'], map_location='cpu'))
model.to(device)

out_path = 'Features_val.pkl'


val_embeddings = []
val_captions = []

input_test = pd.read_csv(config['val_path'])

c = 0
for p in tqdm(input_test.paths):
         #print(p)
         #n= df_eval.iloc[i, 0]#, df_eval.iloc[i, 1]
    text = f'Caption:'
    path = f'{config["video_path"]}{p}'
    try:
        video = read_video(path, transform=None,frames_num=1)

        i = image_grid(video,1,1)
        image = preprocess(i).unsqueeze(0).to(device)

        # image = make_images(video)
        # new_images = []
        # for img in image:
        #     new_images.append(preprocess(img).unsqueeze(0))

        with torch.no_grad():
            prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
            # prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
        val_embeddings.append(prefix)
        val_captions.append(text)
    except Exception as e:
        print(e)

    c+=1
    if c > 50:
        break

answers = []
for i in tqdm(range(len(val_embeddings))):
        emb = val_embeddings[i]

        #qid = df_eval.iloc[i, 2]
        ans = get_ans(emb, 'Caption: ')
        answers.append(ans['answer'])


df = pd.DataFrame({'captions':answers})
df.to_csv('/content/data/answer.csv', index=False)

100%|███████████████████████████████████████| 891M/891M [00:23<00:00, 39.7MiB/s]


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

 98%|█████████▊| 50/51 [00:12<00:00,  2.85it/s]

0.2931036949157715


torch.Size([1, 768])

100%|██████████| 51/51 [00:13<00:00,  3.85it/s]

0.26900315284729004


In [ ]:
path_pred = '/content/data/answer.csv'
df_pred = pd.read_csv(path_pred)
new_preds = []
for x in df_pred['captions']:
    a = ''
    if x is not np.nan:
        for y in x:
            if re.findall(r'[A-Za-z0-9 \.\,\-]', y):
                a += y
    new_preds.append(a)
df_pred['captions'] = new_preds
df_pred.to_csv('/content/data/new_answer.csv', index=False)

In [ ]:
import pandas as pd
import nltk
import numpy as np

path_gt = '/content/data/new_valid.csv'
path_pred = '/content/data/answer.csv'
df_eval = pd.read_csv(path_gt)
df_pred = pd.read_csv(path_pred)
scores = []
for pred, gt in zip(df_eval.caption, df_pred.captions):
    if type(pred)==str and type(gt)==str:
        score = nltk.translate.bleu_score.sentence_bleu([gt.lower().split()], pred.lower().replace('<|endoftext|>','').split(), weights = (0.5, 0.5))
    scores+=[score]
ans = np.array(scores).mean()*100
print(round(ans, 5))

7.56504


In [ ]:
for i in range(50):
    print(df_eval.caption.tolist()[i])
    print(df_pred.captions.tolist()[i])
    print()

Beautiful blue and yellow flowers in an arrangement close-up, pan shot.
Beautiful bouquets of blue and yellow flowers on a blue background.

Dirty car bumper
Winter traffic in the city

Girl running on the street with masks, front view and close to the shot, with the panorama of the street out of focus in the background, with cars, trees, the sidewalk and the sun lighting up.
Girl in a mask running through a park surrounded by trees and buildings, with a stop light in the background on a sunny day.

Woman with a hat watching the ocean
Blue sea and red sky

Married couple receiving at the door of their house another neighbor couple who arrived with welcome gifts, while smiling happily.
A man welcomes his friend into his house while laughing and smiling.

View under a bridge that crosses a river in a big city, where some people go sailing by boat underneath.
People rowing on a bridge over a river surrounded by trees and grass, in a sunny day surrounded by other buildings and buildings wi